# New and Import
## Provenance
* Created new: 2025-06-06

In [1]:
from pathlib import Path 
import pandas as pd
import re
import numpy as np
from functools import partial
from pprint import pprint as pp
from random import sample

import pymupdf # fritz  # PyMuPDF
import Levenshtein

%load_ext autotime
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
time: 34.5 ms (started: 2025-06-07 09:25:23 +01:00)


In [2]:
%autoreload 1
%aimport archivum.mendeley_port
%aimport archivum.library
%aimport archivum.utilities
%aimport archivum.document
%aimport archivum.crossref
%aimport archivum.gui
%aimport archivum.parser
%aimport archivum.reference
%aimport archivum.cli
%aimport archivum.utilities
%aimport archivum.querex

%aimport scratch 
%aimport greater_tables.greater_tables 
%aimport greater_tables.test_tables 
    

import archivum.mendeley_port as arcm
import archivum.library as arcl
import archivum.utilities as arcu
import archivum.document as arcd
import archivum.crossref as arcc
import archivum.gui as arcg
import archivum.parser as arcp
import archivum.reference as arcr
import archivum.cli as cli 

import greater_tables.test_tables as tt
from greater_tables.greater_tables import GT

from archivum.utilities import fGT
import scratch as sc 
%aimport

Modules to reload:
archivum.cli archivum.crossref archivum.document archivum.gui archivum.library archivum.mendeley_port archivum.parser archivum.querex archivum.reference archivum.utilities greater_tables.greater_tables greater_tables.test_tables scratch

Modules to skip:

time: 1.32 s (started: 2025-06-07 09:25:23 +01:00)


In [39]:
lib = arcl.Library('uber-library')
lib.get_library_path_list()

[WindowsPath('C:/Users/steve/AppData/Local/archivum/books.archivum-config'),
 WindowsPath('C:/Users/steve/AppData/Local/archivum/uber-library.archivum-config')]

time: 11.8 ms (started: 2025-06-08 21:09:42 +01:00)


In [38]:
cli.entry(['open-library', 'Uber-Library'], standalone_mode=False)
ans = cli.entry(['new', '-m'], standalone_mode=False)

21:09:35 | archivum.cli.new               | INFO     | Scanning directory C:\Users\steve\Downloads


TypeError: Document.add_meta_data() takes 1 positional argument but 2 were given

time: 495 ms (started: 2025-06-08 21:09:35 +01:00)


In [37]:
df = cli.LibraryContext.last_new
df.head()

AttributeError: type object 'LibraryContext' has no attribute 'last_new'

time: 28.2 ms (started: 2025-06-08 21:09:32 +01:00)


In [7]:
from rich import print

time: 1.7 ms (started: 2025-06-07 21:32:11 +01:00)


In [6]:
df = lib.get_new_documents('', True, False)
df.head()

TypeError: Document.add_meta_data() takes 1 positional argument but 2 were given

time: 754 ms (started: 2025-06-07 09:27:28 +01:00)


In [40]:
d = arcd.Document(df.loc[0, 'path'], lib)

time: 1.97 ms (started: 2025-06-07 09:23:51 +01:00)


In [42]:
d.uber()

AttributeError: 'Document' object has no attribute 'guess_crossref_query'

time: 69.3 ms (started: 2025-06-07 09:24:51 +01:00)


In [ ]:
from rapidfuzz.fuzz import ratio

In [27]:
def best_guess_title(row):
    """Extract best guess for a title using meta data, page 1, and file name."""
    if row.title_similarity > 90:
        return row.meta_title
    elif len(row.meta_title) > len(row.page1_title) and len(row.meta_title) > 10:
        return row.meta_title
    elif len(row.page1_title) > len(row.meta_title) and len(row.page1_title) > 10:
        return row.page1_title
    # no obvious contender
    fn = row.file.replace('.pdf', '').replace('_', ' ')
    if len(fn.split(' ')) > 3:
        return fn
    else:
        return 'UNGUESSABLE'

time: 2.05 ms (started: 2025-06-07 08:53:08 +01:00)


In [29]:
# interesting bits out of new docs
ans = []
for i, row in df.iterrows():
    d = row.Document
    t, a = d.extract_title_author()
    ans.append([row.file_name, row.meta_author, row.meta_author_ex, 
                row.meta_title, t, a])
adf = pd.DataFrame(ans, columns=['file', 'meta_author', 'meta_authorex', 'meta_title', 'page1_title', 'page1_author'])
adf["title_similarity"] = adf.apply(lambda row: ratio(row["meta_title"], row["page1_title"]), axis=1)
adf['best_guess_title'] = adf.apply(best_guess_title, axis=1)

time: 1.1 s (started: 2025-06-07 08:53:55 +01:00)


In [31]:
adf[['file', 'best_guess_title', "meta_title", "page1_title", 'title_similarity' ]].sort_values('title_similarity')[::-1]

,file,best_guess_title,meta_title,page1_title,title_similarity
16,9309211v2.pdf,arXiv:math/9309211v2 [math.FA] 6 Dec 1999,arXiv:math/9309211v2 [math.FA] 6 Dec 1999,arXiv:math/9309211v2 [math.FA] 6 Dec 1999,100.000000
0,s10713-025-00109-z.pdf,Tontines and their modern (re)discovery,Tontines and their modern (re)discovery,Tontines and their modern (re)discovery,97.435897
4,hurst09adf.pdf,Review of automatic document formatting,Review of automatic document formatting,Review of Automatic Document Formatting,92.307692
25,Units of Risk.pdf,Pricing Units of Risk,Units of Risk,Pricing Units of Risk,80.000000
1,Drivers of the Underwriting Cycle Abnormal Re...,Drivers of the Underwriting Cycle: Abnormal Re...,Drivers of the Underwriting Cycle: Abnormal Re...,Return on Property-Liability Insurance Stock and,64.864865
22,CT 12 10738 - Legacy Filing Packet 08-14-2012 ...,Underwriting Profit Provision,CT - Filing Memo.doc,Underwriting Profit Provision,28.000000
26,rpm_2010_handouts_rr3-appel.pdf,Quantifying Risk Load for Property Catastrophe...,Quantifying Risk Load for Property Catastrophe...,Management (RPM) Seminar,22.784810
9,gross-zagier.pdf,Heegner points and derivatives of <Emphasis Ty...,Heegner points and derivatives of <Emphasis Ty...,matbematicae,22.222222
7,Gaussian emergence for stochastic reserve risk...,Presentation title,Presentation title,modelling,22.222222
24,NJ HO 2024 Filing.pdf,"to arrive at Non-Tenant Homeowners, Renters, a...",@NJ HO 2024 Filing - Memo.doc,"to arrive at Non-Tenant Homeowners, Renters, a...",20.312500


time: 10.2 ms (started: 2025-06-07 08:54:15 +01:00)


In [ ]:
def get_crossref_info()

In [32]:
# best guess title
bgt = adf.loc[4, 'best_guess_title']
bgt

'Review of automatic document formatting'

time: 4.87 ms (started: 2025-06-07 08:55:06 +01:00)


In [33]:
cr1 = arcc.search_by_title(bgt)

time: 488 ms (started: 2025-06-07 08:55:10 +01:00)


In [35]:
type(cr1)

dict

time: 3.82 ms (started: 2025-06-07 08:56:01 +01:00)


In [36]:
ref1 = arcr.Reference.from_crossref(cr1)
ref1

Reference(tag='a6a19efaf5eddbfb4ea5291323405ff22025', type='inproceedings', title='Review of automatic document formatting', year=2009, author='Hurst, Nathan and Li, Wilmot and Marriott, Kim', journal=None, volume=None, number=None, month=None, pages='99-108', booktitle='Proceedings of the 9th ACM symposium on Document engineering', editor=None, edition=None, chapter=None, doi='10.1145/1600193.1600217', isbn=None, publisher='ACM', institution=None, address=None, url='https://doi.org/10.1145/1600193.1600217', mendeley_tags=None, arc_citations=None, arc_source='import', ref_source='crossref')

time: 3.7 ms (started: 2025-06-07 08:56:09 +01:00)


In [38]:
GT(ref1.to_ref_ser())

index,0
tag,a6a19efaf5eddbfb4ea5291323405ff22025
type,inproceedings
title,Review of automatic document formatting
year,"2,009"
author,"Hurst, Nathan and Li, Wilmot and Marriott, Kim"
journal,
volume,
number,
month,
pages,99-108


time: 30.7 ms (started: 2025-06-07 08:57:02 +01:00)


# RICH TABLE

In [11]:
from greater_tables import GT
import archivum.library as arcl
from rich.console import Console
from rich import box, inspect

lib = arcl.Library('uber-library')
bit = lib.ref_df.head().iloc[:, :9].drop(columns=['pages','doi', 'volume'])
bit

,tag,type,author,journal,title,year
0,Schmidt2003,article,"Schmidt, Anatoly B.",Physica A: Statistical Mechanics and its Appli...,{The true invariant of an arbitrage free portf...,2003
1,Bahnemann2015,book,"Bahnemann, David",,{Distributions for Actuaries},2015
2,Khoshnevisan2011,techreport,"Khoshnevisan, Davar",,{Topics in Probability: Levy Processes},2011
3,Mahul2003,article,"Mahul, Olivier",Paper presented at the 2003 NBER Insurance Pro...,{Efficient Risk Sharing within a Catastrophe I...,2003
4,Jiang2025,article,"Jiang, By John X. and Koo, David S. and Wang, ...",The International Journal of Accounting,{The Road Not Taken: Evaluating the Consequenc...,2025


In [15]:
df = lib.ref_df.head()
df

,tag,type,author,doi,journal,pages,title,volume,year,publisher,...,institution,number,mendeley-tags,booktitle,edition,month,address,editor,arc-citations,arc-source
0,Schmidt2003,article,"Schmidt, Anatoly B.",10.1016/S0378-4371(02)01551-0,Physica A: Statistical Mechanics and its Appli...,535--538,{The true invariant of an arbitrage free portf...,320,2003,,...,,,,,,,,,0,mendeley
1,Bahnemann2015,book,"Bahnemann, David",,,,{Distributions for Actuaries},,2015,Casualty Actuarial Society Mongraphs No. 2,...,,,,,,,,,0,mendeley
2,Khoshnevisan2011,techreport,"Khoshnevisan, Davar",,,,{Topics in Probability: Levy Processes},,2011,,...,University of Utah,,,,,,,,0,mendeley
3,Mahul2003,article,"Mahul, Olivier",,Paper presented at the 2003 NBER Insurance Pro...,,{Efficient Risk Sharing within a Catastrophe I...,,2003,,...,,October,,,,,,,0,mendeley
4,Jiang2025,article,"Jiang, By John X. and Koo, David S. and Wang, ...",,The International Journal of Accounting,,{The Road Not Taken: Evaluating the Consequenc...,,2025,,...,,,PMIR,,,,,,0,mendeley


In [12]:
console = Console(force_jupyter=True)

In [3]:
f = GT(bit, show_index=False, max_table_width=120, year_cols=['year'], hrule_widths=1)
console.print(f.rich_table(console))

┌───────────────────┬────────────┬─────────────────────┬──────────────────────┬────────────────────────────┬──────┐
│ tag               │ type       │ author              │ journal              │ title                      │ year │
├───────────────────┼────────────┼─────────────────────┼──────────────────────┼────────────────────────────┼──────┤
│                   │            │                     │ Physica A:           │                            │      │
│ Schmidt2003       │ article    │ Schmidt, Anatoly B. │ Statistical          │ {The true invariant of an  │ 2003 │
│                   │            │                     │ Mechanics and its    │ arbitrage free portfolio}  │      │
│                   │            │                     │ Applications         │                            │      │
├───────────────────┼────────────┼─────────────────────┼──────────────────────┼────────────────────────────┼──────┤
│ Bahnemann2015     │ book       │ Bahnemann, David    │                      │ {Distributions for         │ 2015 │
│                   │            │                     │                      │ Actuaries}                 │      │
├───────────────────┼────────────┼─────────────────────┼──────────────────────┼────────────────────────────┼──────┤
│ Khoshnevisan2011  │ techreport │ Khoshnevisan, Davar │                      │ {Topics in Probability:    │ 2011 │
│                   │            │                     │                      │ Levy Processes}            │      │
├───────────────────┼────────────┼─────────────────────┼──────────────────────┼────────────────────────────┼──────┤
│                   │            │                     │ Paper presented at   │ {Efficient Risk Sharing    │      │
│ Mahul2003         │ article    │ Mahul, Olivier      │ the 2003 NBER        │ within a Catastrophe       │ 2003 │
│                   │            │                     │ Insurance Project    │ Insurance Pool}            │      │
│                   │            │                     │ Workshop             │                            │      │
├───────────────────┼────────────┼─────────────────────┼──────────────────────┼────────────────────────────┼──────┤
│                   │            │                     │                      │ {The Road Not Taken:       │      │
│                   │            │ Jiang, By John X.   │ The International    │ Evaluating the             │      │
│ Jiang2025         │ article    │ and Koo, David S.   │ Journal of           │ Consequences of the U.S.'s │ 2025 │
│                   │            │ and Wang, Isabel Y. │ Accounting           │ Non-Adoption of IFRS in    │      │
│                   │            │                     │                      │ 2012}                      │      │
└───────────────────┴────────────┴─────────────────────┴──────────────────────┴────────────────────────────┴──────┘

In [8]:
import pandas as pd
import numpy as np

In [10]:
    df = pd.DataFrame([
        {"name": "Alice", "role": "Analyst", "city": "London", "status": "Active" },
        {"name": "Bob", "role": "Manager", 'level': 3, "city": "Paris", "status": "Inactive", "notes": "On leave"},
        {"name": "Carol", "role": np.nan, "city": "Berlin", "status": "Active", "notes": ""},
    ])
    df

,name,role,city,status,level,notes
0,Alice,Analyst,London,Active,NaN,NaN
1,Bob,Manager,Paris,Inactive,3.0,On leave
2,Carol,NaN,Berlin,Active,NaN,
